In [2]:
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.preprocessing import image

import matplotlib.pyplot as plt
%matplotlib inline

import models.simpleConv as simpleConv


dataPath = 'RealImageNet/ImageNetSubsample/Data/CLS-LOC'
trainPath = os.path.join(dataPath, 'train')
# testPath = os.path.join(dataPath, 'test')
# validationPath = os.path.join(dataPath, 'val')

img_width, img_height = 224, 224

Using TensorFlow backend.


In [3]:
# from keras.preprocessing import image
# get training data
x_train = []
y_train = []
trainingFolders = [x for x in os.listdir(trainPath)]

lines = [line.rstrip('\n').split() for line in open('RealImageNet/LOC_synset_mapping.txt')]
wnids_to_words = {line[0]:' '.join(line[1:]) for line in lines }

train_bboxes = {} #{image_name: list(bboxes)}
for id in trainingFolders:
    boxesPath = os.path.join("RealImageNet", "LOC_train_solution.csv")
    lines = [line.rstrip('\n').split(',') for line in open(boxesPath)][1:]
    for line in lines:
        imageId = line[0]
        predictionString = line[1]
        split = predictionString.split()
        train_bboxes[imageId] = []
        for i in range(0, len(split), 5):
            box = split[i:i+5]
            train_bboxes[imageId].append(box)

for imageName in train_bboxes.keys():
    imageLabel = imageName.split('_')[0]
    imagePath = os.path.join(trainPath, imageLabel, imageName + ".JPEG") #folder name
    
    if os.path.exists(imagePath):
        img = image.load_img(imagePath, target_size=(224, 224)) #pil format
        x = image.img_to_array(img) 
#         x = np.expand_dims(x, axis=0) # making a numpy array (surrounds with another lis)
#         x_train.append(x/255.)
        for box in train_bboxes[imageName]:
            x_train.append(x/255.)
            idk = [imageLabel[1:]]
            y_train.append(idk)




x_train = np.array(x_train)
# x_train = np.expand_dims(x_train, axis= 1)
y_train = np.array(y_train, dtype = 'uint')
# y_train = np.expand_dims(y_train, axis= 0)


from sklearn.preprocessing import LabelBinarizer
label_encoder = LabelBinarizer()
y_train = label_encoder.fit_transform(y_train)

In [6]:
y_train

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [7]:
model = simpleConv.build_model(img_width, img_height, 3)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1, verbose=1)

Epoch 1/1
 160/2948 [>.............................] - ETA: 3:36 - loss: 1.6424 - acc: 0.3000

KeyboardInterrupt: 